# Main results notebook

this notebook will allow you to rerun the main statistics for the reported effects in the paper

the code for this is found under src.utils.make_primary_stats, I have copied the functions here so that you may examine them before running them

In [1]:
import numpy as np
import pandas as pd
import src.utils.make_primary_stats as stats
import src.utils.dataset_utils as ds_utils
import pingouin as pg

datadir = '/mnt/data/processed/experiment_2'

In [2]:
data_dict = ds_utils.load_exp_data(datadir)

/mnt/src/utils/dataset_utils.py:584: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  good_stakes.groupby(["subid"])["points", "rews_together", "rt_1", "rt_2"]


In [3]:
def model_fitting_descriptive_stats(w1_map_df):
    print("Summary stats for different components of model")
    print(
        w1_map_df[
            [
                "beta fit",
                "alpha fit",
                "lambda fit",
                "w low stakes low arm fit",
                "stickiness fit",
                "resp stickiness fit",
                "eta fit",
                "kappa fit",
                "LL",
            ]
        ].mean()
    )
    print("")

In [4]:
stats.model_fitting_descriptive_stats(data_dict['w1_map_df'])

Summary stats for different components of model
beta fit                      1.514586
alpha fit                     0.648679
lambda fit                    0.529495
w low stakes low arm fit      0.571975
stickiness fit                0.258882
resp stickiness fit          -0.289249
eta fit                       0.484691
kappa fit                     0.502964
LL                          143.861729
dtype: float64



You can see the average $w$ parameter fit is 0.572

---
## Behavioral indices of representational change track task structure.

In [5]:
print(f"including {data_dict['model_mat_fits'].subid.nunique()} subjects")
print("1 sample ttest from 0 for Visual Co-occurence fit")
print(pg.ttest(data_dict['model_mat_fits']["Visual cooccurrence"], 0))
print("")
print("1 sample ttest from 0 for Direct reward association fit")
print(pg.ttest(data_dict['model_mat_fits']["Direct item association"], 0))
print("")
print("1 sample ttest from 0 for Indirect outcome-linked association fit")
print(pg.ttest(data_dict['model_mat_fits']["Indirect item association"], 0))
print("")

including 161 subjects
1 sample ttest from 0 for Visual Co-occurence fit
              T  dof alternative     p-val          CI95%   cohen-d      BF10  \
T-test  4.63838  160   two-sided  0.000007  [4.48, 11.12]  0.365556  1802.332   

          power  
T-test  0.99598  

1 sample ttest from 0 for Direct reward association fit
               T  dof alternative         p-val          CI95%   cohen-d  \
T-test  5.979339  160   two-sided  1.413007e-08  [7.15, 14.21]  0.471238   

             BF10  power  
T-test  6.929e+05    1.0  

1 sample ttest from 0 for Indirect outcome-linked association fit
               T  dof alternative         p-val          CI95%   cohen-d  \
T-test  5.286294  160   two-sided  4.036660e-07  [5.39, 11.82]  0.416618   

           BF10     power  
T-test  2.8e+04  0.999507  



---
## Representations of task structure correlate with task performance and model-based control.

### Correlation of model matrix fits and baseline-correct points earned in the decision-making task

In [6]:
def model_mat_fits_points_corr(model_mat_fits, grouped_stakes):
    model_mat_points = pd.merge(model_mat_fits, grouped_stakes, on="subid")
    melted_mmp = model_mat_points[
        [
            "subid",
            "Visual cooccurrence",
            "Direct item association",
            "Indirect item association",
            "Points earned in decision-making task",
            "rt_2",
        ]
    ].melt(
        id_vars=["subid", "Points earned in decision-making task", "rt_2"],
        var_name="Grouping Model",
        value_name=r"Model Matrix Fit ($\bar{\beta}$)",
    )
    melted_mmp["model_matrix_fit"] = melted_mmp[r"Model Matrix Fit ($\bar{\beta}$)"]
    print("")
    print(f"including {melted_mmp.subid.nunique()} subjects")
    print(
        "Correlation of Visual Co-occurence β with Points earned in decision-making task"
    )
    print(
        pg.corr(
            melted_mmp[melted_mmp["Grouping Model"] == "Visual cooccurrence"][
                "Points earned in decision-making task"
            ],
            melted_mmp[melted_mmp["Grouping Model"] == "Visual cooccurrence"][
                "model_matrix_fit"
            ],
        )
    )
    print(
        "Correlation of Direct reward association β with Points earned in decision-making task"
    )
    print(
        pg.corr(
            melted_mmp[melted_mmp["Grouping Model"] == "Direct item association"][
                "Points earned in decision-making task"
            ],
            melted_mmp[melted_mmp["Grouping Model"] == "Direct item association"][
                "model_matrix_fit"
            ],
        )
    )
    print("")
    print(
        "Correlation of Indirect outcome-linked association β with Points earned in decision-making task"
    )
    print(
        pg.corr(
            melted_mmp[melted_mmp["Grouping Model"] == "Indirect item association"][
                "Points earned in decision-making task"
            ],
            melted_mmp[melted_mmp["Grouping Model"] == "Indirect item association"][
                "model_matrix_fit"
            ],
        )
    )
    print("")

In [7]:
stats.model_mat_fits_points_corr(data_dict['model_mat_fits'], data_dict['grouped_stakes'])


including 161 subjects
Correlation of Visual Co-occurence β with Points earned in decision-making task
           n        r          CI95%     p-val   BF10     power
pearson  161  0.00377  [-0.15, 0.16]  0.962143  0.099  0.050159
Correlation of Direct reward association β with Points earned in decision-making task
           n         r         CI95%     p-val     BF10     power
pearson  161  0.324501  [0.18, 0.46]  0.000027  614.596  0.988808

Correlation of Indirect outcome-linked association β with Points earned in decision-making task
           n         r         CI95%         p-val       BF10     power
pearson  161  0.445976  [0.31, 0.56]  3.050133e-09  3.603e+06  0.999978



In [8]:
def model_mat_fits_w_corr(model_mat_fits, w1_map_df):
    melted_1w_map = w1_map_df[["subid", "w low stakes low arm fit"]].rename(
        columns={"w low stakes low arm fit": "w fit"}
    )
    model_mat_1w_map = pd.merge(model_mat_fits, melted_1w_map, on="subid")
    mm1w_melt = model_mat_1w_map.melt(
        id_vars=["subid", "high_arm", "w fit"],
        var_name="Grouping Model",
        value_name=r"Model Matrix Fit ($\bar{\beta}$)",
    )
    mm1w_melt["model_matrix_fit"] = mm1w_melt[r"Model Matrix Fit ($\bar{\beta}$)"]
    print(f"including {mm1w_melt.subid.nunique()} subjects")
    print("Correlation of Visual Co-occurrence β with w fit")
    print(
        pg.corr(
            mm1w_melt[mm1w_melt["Grouping Model"] == "Visual cooccurrence"]["w fit"],
            mm1w_melt[mm1w_melt["Grouping Model"] == "Visual cooccurrence"][
                "model_matrix_fit"
            ],
        )
    )

    print("")
    print("Correlation of Direct reward association β with w fit")
    print(
        pg.corr(
            mm1w_melt[mm1w_melt["Grouping Model"] == "Direct item association"][
                "w fit"
            ],
            mm1w_melt[mm1w_melt["Grouping Model"] == "Direct item association"][
                "model_matrix_fit"
            ],
        )
    )

    print("")
    print("Correlation of Indirect outcome-linked association β with w fit")
    print(
        pg.corr(
            mm1w_melt[mm1w_melt["Grouping Model"] == "Indirect item association"][
                "w fit"
            ],
            mm1w_melt[mm1w_melt["Grouping Model"] == "Indirect item association"][
                "model_matrix_fit"
            ],
        )
    )

    print("")

### Correlation of model matrix fits and model-based control parameter 

In [9]:
stats.model_mat_fits_w_corr(data_dict['model_mat_fits'], data_dict['w1_map_df'])

including 161 subjects
Correlation of Visual Co-occurrence β with w fit
           n         r          CI95%     p-val   BF10     power
pearson  161 -0.140595  [-0.29, 0.01]  0.075257  0.473  0.430136

Correlation of Direct reward association β with w fit
           n         r         CI95%     p-val     BF10     power
pearson  161  0.325581  [0.18, 0.46]  0.000025  653.687  0.989248

Correlation of Indirect outcome-linked association β with w fit
           n         r         CI95%         p-val       BF10     power
pearson  161  0.477897  [0.35, 0.59]  1.448315e-10  6.852e+07  0.999998



---
## Motivation affects representational change.

### Stake x Arm ANOVA results

In [10]:
def model_based_condition_anova(w4_map_df):
    melted_4w_map = w4_map_df[
        [
            "subid",
            "w low stakes low arm fit",
            "w high stakes low arm fit",
            "w low stakes high arm fit",
            "w high stakes high arm fit",
        ]
    ].melt(id_vars=["subid"], var_name="type of", value_name="w")
    melted_4w_map["arm"] = "high"
    melted_4w_map.loc[
        np.where(melted_4w_map["type of"] == "w low stakes low arm fit")[0], "arm"
    ] = "low"
    melted_4w_map.loc[
        np.where(melted_4w_map["type of"] == "w high stakes low arm fit")[0], "arm"
    ] = "low"

    melted_4w_map["stakes"] = "5"
    melted_4w_map.loc[
        np.where(melted_4w_map["type of"] == "w low stakes low arm fit")[0], "stakes"
    ] = "1"
    melted_4w_map.loc[
        np.where(melted_4w_map["type of"] == "w low stakes high arm fit")[0], "stakes"
    ] = "1"
    aov = pg.rm_anova(
        dv="w",
        within=["arm", "stakes"],
        subject="subid",
        data=melted_4w_map,
        detailed=True,
    )
    # Pretty printing of ANOVA summary
    pg.print_table(aov)
    print(
        f"5x w parameter is {melted_4w_map.loc[np.where(melted_4w_map['stakes'] == '5'), 'w'].mean()}"
    )
    print(
        f"1x w parameter is {melted_4w_map.loc[np.where(melted_4w_map['stakes'] == '1'), 'w'].mean()}"
    )

In [11]:
stats.model_based_condition_anova(data_dict['w4_map_df'])


ANOVA SUMMARY

Source           SS    ddof1    ddof2     MS      F    p-unc    p-GG-corr    np2    eps
------------  -----  -------  -------  -----  -----  -------  -----------  -----  -----
arm           0.015        1      160  0.015  1.346    0.248        0.248  0.008  1.000
stakes        0.091        1      160  0.091  7.265    0.008        0.008  0.043  1.000
arm * stakes  0.024        1      160  0.024  1.286    0.258        0.258  0.008  1.000

5x w parameter is 0.5645616887045466
1x w parameter is 0.5408204669090588


---
### 1 sample t-tests for the difference in beta values between high and low stake environment versions of the behRSA regressions

In [12]:
print(f"including {data_dict['model_mat_hvl'].subid.nunique()} subjects")
print("1 sample ttest from 0 for state1_diff")
print(pg.ttest(data_dict['model_mat_hvl']["state1_diff"], 0))
print("")
print("1 sample ttest from 0 for l_diff")
print(pg.ttest(data_dict['model_mat_hvl']["l_diff"], 0))
print("")
print("1 sample ttest from 0 for o_diff")
print(pg.ttest(data_dict['model_mat_hvl']["o_diff"], 0))
print("")

including 161 subjects
1 sample ttest from 0 for state1_diff
               T  dof alternative     p-val          CI95%   cohen-d   BF10  \
T-test  0.452059  160   two-sided  0.651839  [-2.23, 3.56]  0.035627  0.097   

           power  
T-test  0.073435  

1 sample ttest from 0 for l_diff
               T  dof alternative    p-val          CI95%   cohen-d   BF10  \
T-test  1.740524  160   two-sided  0.08369  [-0.23, 3.69]  0.137173  0.384   

           power  
T-test  0.409202  

1 sample ttest from 0 for o_diff
               T  dof alternative     p-val         CI95%   cohen-d    BF10  \
T-test  3.222347  160   two-sided  0.001541  [1.75, 7.29]  0.253957  12.354   

           power  
T-test  0.893063  



---
## Memory for object-background pairings is better in higher-stakes contexts.

### $d'$ differences from high and low-stakes environments

In [13]:
def memory_dprime_ttest(dprime_df):
    dprime_stat_df = dprime_df.groupby(["subid"]).first().reset_index()
    print(f"including {dprime_df.subid.nunique()} subjects")
    print("paired sample t-test between dprime mismatch for high and low arm")
    print(
        pg.ttest(
            dprime_stat_df["dprime_lbst_high"],
            dprime_stat_df["dprime_lbst_low"],
            paired=True,
        )
    )
    print("")
    print("paired sample t-test between dprime lure for high and low arm")
    print(
        pg.ttest(
            dprime_stat_df["dprime_lt_high"],
            dprime_stat_df["dprime_lt_low"],
            paired=True,
        )
    )
    print("")

In [14]:
stats.memory_dprime_ttest(data_dict['dprime_df'])

including 161 subjects
paired sample t-test between dprime mismatch for high and low arm
               T  dof alternative     p-val         CI95%   cohen-d   BF10  \
T-test  2.289169  160   two-sided  0.023377  [0.02, 0.31]  0.167201  1.108   

          power  
T-test  0.55918  

paired sample t-test between dprime lure for high and low arm
               T  dof alternative     p-val         CI95%   cohen-d   BF10  \
T-test  0.816886  160   two-sided  0.415208  [-0.1, 0.25]  0.076586  0.122   

           power  
T-test  0.161824  

